# Calculate the correlations for each layer and create input RDMs

## Set up the environment

In [ ]:
import numpy as np
import os
import json
import h5py
import tables

## Define functions to create filenames

In [ ]:
def getFileName(n_samples, name):
    return name \
        + "_{}_".format(n_samples) \
        + "_{}_".format(model_name) \
        + "_{}".format(layer_name)  \
        + ".npy"       

## Get the size of the activations

In [ ]:
#load the np file containing the shape of the activations
ROOT_PATH = '/mnt/raid/ni/agnessa/RSA/'
NR_OF_SAMPLES = 10000
json_file_layers=os.path.join(ROOT_PATH,'resnets_selected_layers.json')
with open(json_file_layers, "r") as fd:
    selected_layers = json.load(fd)
model_name, layer_name = selected_layers[17].get('model'),selected_layers[17].get('layer') #change the index at each iteration     
path = os.path.join(ROOT_PATH + 'activations/', getFileName(NR_OF_SAMPLES, "shape"))
activations_shape = np.load(path)


## Define the correlation function

In [ ]:
# my version
def correlationd_matrix(batch_size): #(list_of_activations, n) ,array_activations
    total_size,activation_length = activations_shape
    print("Activation Length: ", activation_length)
    print("Total Size: ", total_size)

    #memmap is used to access a part of the file 
    file_name = os.path.join(ROOT_PATH+'activations/',getFileName(NR_OF_SAMPLES,'activations'))
    act = np.memmap(file_name, mode="r", shape=(total_size, activation_length)) #numsamples x numfeatures
    correlationd = np.zeros((total_size,total_size))
    correlationd[:] = np.nan
    num_batches = int(total_size / batch_size)
#     total = sum(x+1 for x in range(num_batches))-num_batches #num 1000-wise comparisons to do: 55-(comparisons of the same sample) = 45
    total = sum(x+1 for x in range(num_batches)) #num 1000-wise comparisons to do: 55
    index = 0
   
    for i in range(num_batches):  #num_batches-1 
        start_1 = batch_size*i
        end_1 = batch_size*(i+1)
        list_of_activations_1 = act[start_1:end_1,:]

        for j in range(num_batches-i): #(i+1,num_batches): 
            index += 1
            print("New Iteration: i = {0}, j = {1}; {2}/{3}".format(i,j,index,total))
            start_2 = batch_size*(i+j)
            end_2 = batch_size*(i+j+1)
            list_of_activations_2 = act[start_2:end_2,:]
            corr_activations = 1-np.corrcoef(list_of_activations_1,list_of_activations_2) #2000 x 2000 matrix
            for x in range(corr_activations.shape[0]):
                for y in range(corr_activations.shape[1]):
                    if x < batch_size:
                        start_x = start_1
                    else: 
                        start_x = start_2-1000
                    if y < batch_size:
                        start_y = start_1
                    else:
                        start_y = start_2-1000                       
                    correlationd[x+start_x,y+start_y] = correlationd[y+start_y,x+start_x] = corr_activations[x,y]
        
    return(correlationd)

In [ ]:
print('Calculating the correlations for model: ',model_name,'and layer: ',layer_name)
corr_matrix= correlationd_matrix(1000) 
path = os.path.join(ROOT_PATH + 'Input_RDM/', getFileName(NR_OF_SAMPLES, "Input_RDM"))
print("Save Input RDM -> {}".format(path))
np.save(path, np.array(corr_matrix)) 

In [ ]:
# Felix's version

def correlationd_matrix(batch_size): #(list_of_activations, n) ,array_activations
    total_size,activation_length = activations_shape
    print("Activation Length: ", activation_length)
    print("Total Size: ", total_size)
    
    def pearsonr_optimized(xm, ss_xm, ym, ss_ym):
#         x = np.asarray(x)
#         y = np.asarray(y)
#         n = len(x)
#         mx = x.mean()
#         my = y.mean()
#         xm, ym = x - mx, y - my
        r_num = np.add.reduce(xm * ym)
        r_den = np.sqrt(ss_xm * ss_ym)
        r = r_num / r_den

        # Presumably, if abs(r) > 1, then it is only some small artifact of floating
        # point arithmetic.
        r = max(min(r, 1.0), -1.0)

        return r

    #copied directly from scipy sources without change 
    def ss(a, axis=0):
        def _chk_asarray(a, axis):
            if axis is None:
                a = np.ravel(a)
                outaxis = 0
            else:
                a = np.asarray(a)
                outaxis = axis

            if a.ndim == 0:
                a = np.atleast_1d(a)

            return a, outaxis

        a, axis = _chk_asarray(a, axis)
        return np.sum(a*a, axis)

    #memmap is used to access a part of the file. maybe: use memmap in the loop to access the first 1000 samples?    
    file_name = os.path.join(ROOT_PATH+'activations/',getFileName(NR_OF_SAMPLES,'activations'))
    act = np.memmap(file_name, mode="r", shape=(total_size, activation_length)) #numsamples x numfeatures
    correlationd = np.zeros((total_size,total_size))
    correlationd[:] = np.nan
    total = sum(x+1 for x in range(int(total_size / batch_size))) #num 1000-wise comparisons to do: 55
    index = 0
    
    for i in range(int(total_size / batch_size)):
        centered_activations_1 = np.ones((batch_size,activation_length)) * -17
        centered_squared_summed_activations_1 = np.ones((batch_size,)) * -17

        start_1 = batch_size*i
        end_1 = batch_size*(i+1)

        list_of_activations_1 = act[start_1:end_1,:]

        for j in range(int(total_size / batch_size)-i):
            index += 1
            print("New Iteration: i = {0}, j = {1}; {2}/{3}".format(i,j,index,total))

            start_2 = batch_size*(i+j)
            end_2 = batch_size*(i+j+1)

            list_of_activations_2 = act[start_2:end_2,:]

            centered_activations_2 = np.ones((batch_size,activation_length)) * -17
            centered_squared_summed_activations_2 = np.ones((batch_size,)) * -17


            for k in range(batch_size):
                if k % 200 == 0:
                    print("Centering... done {0} of {1}".format(k, batch_size))
                centered_activations_1[k] = list_of_activations_1[k] - list_of_activations_1[k].mean()
                centered_squared_summed_activations_1[k] = ss(centered_activations_1[k])

                centered_activations_2[k] = list_of_activations_2[k] - list_of_activations_2[k].mean()
                centered_squared_summed_activations_2[k] = ss(centered_activations_2[k])


            for l in range(batch_size):
                if l % 200 == 0:
                    print("Correlation... done {0} of {1}".format(l, batch_size))
                for m in range(batch_size):
                    correlationd[start_1+l,start_2+m] = correlationd[start_2+m, start_1+l] = 1 - pearsonr_optimized(centered_activations_1[l], centered_squared_summed_activations_1[l], centered_activations_2[m], centered_squared_summed_activations_2[m])

    return(correlationd)



## In case the load function does not work

In [ ]:
#If the load function gives an error, do this
np_load_old = np.load # modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
activations_shape = np.load(path)
np.load = np_load_old